In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Load chunks.txt and store it in a list, separated by newlines
with open('chunks.txt', 'r') as f:
    chunks = f.read().split('\n')


In [3]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
import os
import dotenv

dotenv.load_dotenv('env')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [6]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain" # put in the name of your pinecone index here

In [9]:
#docsearch = Pinecone.from_texts(chunks, embeddings, index_name=index_name) #Initial population !!
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [10]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [11]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [36]:
query = """" Recomend a game like this:
Stealth based game assassin game. Not assassins creed. Good 3d graphics.."""
docs = docsearch.similarity_search(query, include_metadata=True)
docs

[Document(page_content='Assassin\'s Creed® Syndicate: ---{Graphics}---  You forget what reality is  Beautiful  Good  Decent  Bad  Dont look too long at it  MS-DOS  ---{Gameplay}---  Very good  Good  Its just gameplay  Mehh  Watch paint dry instead  Just don\'t  ---{Audio}---  Eargasm  Very good  Good  Not too bad  Bad  I\'m now deaf  ---{Audience}---  Kids  Teens  Adults  All  ---{PC Requirements}---  Check if you can run paint  Potato  Decent  Fast  Rich boi  Ask NASA if they have a spare computer  ---{Difficulity}---  Just press \'W\'  Easy  Significant brain usage  Easy to learn / Hard to master  Difficult  Dark Souls  ---{Grind}---  Nothing to grind  Only if u care about leaderboards/ranks  Isnt necessary to progress  Average grind level  Too much grind  Youll need a second live for grinding  ---{Story}---  No Story  Some lore  Average  Good  Lovely  Itll replace your life  ---{Game Time}---  Long enough for a cup of coffee  Short  Average  Long  To infinity and beyond  ---{Price}-

In [37]:
chain.run(input_documents=docs, question=query)

' Deceive Inc. is a great stealth-based game that is similar to an assassin game. It has good 3D graphics and offers unique gameplay with short, fast-paced matches. There is a good variety of characters with different weapons and abilities, so everyone can find something to their liking. It is great fun with friends, although it does not have a duo mode. Highly recommend you give this game a shot!'